<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/GPT2_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
%tensorflow_version 1.x
#!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
import pandas as pd
speeches = pd.read_pickle("search_dataset_111.pkl")

# make train and eval split
speeches_train = speeches.iloc[:-10000]
speeches_eval = speeches.iloc[-10000:]
print(len(speeches_train), len(speeches_eval))

45439 10000


In [ ]:
speeches_train.head()

,speech,party,stemmed_speech,year
876114,Our democracy renews itself every 2 years as M...,D,our democraci renew itself everi 2 year as mem...,111
876115,"Madam Speaker. Leader HOYER, fellow Members, a...",R,"madam speaker . leader hoyer , fellow member ,...",111
876116,"Thank you very much. Leader BOEHNER. Together,...",D,thank you veri much . leader boehner . togeth ...,111
876117,for your nomination this morning. Thank you to...,D,for your nomin thi morn . thank you to the mem...,111
876118,"Mr. Speaker, for purposes of debate only, I yi...",D,"mr. speaker , for purpos of debat onli , i yie...",111


In [ ]:
# make separate txt files for republicans and democrats, both for train and eval
with open("rep_speeches_train.txt", 'w') as f1, open("dem_speeches_train.txt", 'w') as f2:
  for i in range(len(speeches_train)):
    speech = speeches_train.iloc[i].speech
    party = speeches_train.iloc[i].party
    if party == 'R':
      f1.write(speech + '\n')
    else:
      f2.write(speech + '\n')

with open("rep_speeches_eval.txt", 'w') as f1, open("dem_speeches_eval.txt", 'w') as f2:
  for i in range(len(speeches_eval)):
    speech = speeches_eval.iloc[i].speech
    party = speeches_eval.iloc[i].party
    if party == 'R':
      f1.write(speech + '\n')
    else:
      f2.write(speech + '\n')

In [ ]:
# check files 
with open('rep_speeches_train.txt', 'r') as f:
  lines = f.readlines()
  for line in lines[:5]:
    print(line)

Madam Speaker. Leader HOYER, fellow Members, and a special welcome to our new Members and their families and friends who are here today. We begin this new Congress at a great time of challenge for the American people. This winter, working families are struggling to pay their bills and keep their homes, small businesses are being forced to choose between cutting jobs and closing their doors, health costs are rising, college savings funds and 401(k)s have declined in value substantially, parents are deeply worried about their childrens future, I think its a time of anxiety for millions of Americans, some of whom face economic challenges not seen in this country for generations. When things are at their worst for the American people, we owe them our best. This Congress must rise to the occasion. Two weeks from today, we will inaugurate a new President. Presidentelect Obama has expressed a desire to govern from the center and put the needs of our country first, I think all of you know Wash

####Generate without fine tuning

In [ ]:
import logging
from simpletransformers.language_generation import LanguageGenerationModel

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model = LanguageGenerationModel("gpt2", "gpt2", args={"length": 256})

prompts = [
   "What do you believe about tax increases?",
   "Should abortion be illegal?",
   "Is climate change real?",
   "Should we have higher taxes for higher incomes?"
]

for prompt in prompts:
    generated = model.generate(prompt, verbose=False)

    #generated = '.'.join(generated[0].split('.')[:-1]) + '.'
    print(generated)

You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


['What do you believe about tax increases?\n\nThe tax increases are already happening because Congress and the president agreed on the debt ceiling increase.']


['Should abortion be illegal? (The Supreme Court has not heard any arguments on such a case.)\n\nIn 2006, the']


['Is climate change real?\n\n"It\'s so hard, the climate is changing for so long, they\'re not really']
['Should we have higher taxes for higher incomes?\n\nI do not believe that this is a one way or the other of raising taxes to pay']


###Fine tuning

In [ ]:
gpt2.download_gpt2(model_name="124M")

Fetching checkpoint: 1.05Mit [00:00, 476Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 15.5Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 745Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:03, 131Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 342Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 143Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 172Mit/s]                                                       


In [ ]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pwd

/content


In [ ]:
file_name = "rep_speeches_train.txt"

In [ ]:
gpt2.copy_file_from_gdrive(file_name)

In [ ]:
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run1',
              print_every=10,
              sample_every=200,
              save_every=500
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:33<00:00, 33.94s/it]


dataset has 7245447 tokens
Training...
[10 | 27.91] loss=3.09 avg=3.09
[20 | 49.52] loss=3.07 avg=3.08
[30 | 71.34] loss=3.06 avg=3.07
[40 | 93.36] loss=2.94 avg=3.04
[50 | 115.48] loss=3.00 avg=3.03
[60 | 137.78] loss=2.87 avg=3.00
[70 | 160.24] loss=3.09 avg=3.02
[80 | 182.79] loss=2.96 avg=3.01
[90 | 205.42] loss=2.98 avg=3.01
[100 | 228.10] loss=2.92 avg=3.00
[110 | 250.85] loss=2.85 avg=2.98
[120 | 273.64] loss=2.93 avg=2.98
[130 | 296.47] loss=3.01 avg=2.98
[140 | 319.34] loss=3.07 avg=2.99
[150 | 342.22] loss=2.94 avg=2.98
[160 | 365.14] loss=2.92 avg=2.98
[170 | 388.07] loss=2.93 avg=2.98
[180 | 411.03] loss=2.94 avg=2.97
[190 | 434.05] loss=2.78 avg=2.96
[200 | 457.09] loss=2.96 avg=2.96
======== SAMPLE 1 ========
 of Congress, I am also pleased that my colleague from Maryland is in the process of submitting a letter to the other ranking members, which says: I am not satisfied with the approach proposed by the gentleman from Louisiana and the gentleman from Maryland. And I loo

In [ ]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="Is climate change real?",
              nsamples=1,
              batch_size=1
              )

Is climate change real? Is it a category higher than the one that is a national security threat? Are we going to say we are addressing climate change? Are we going to say we are raising the cap that will be on carbon emissions, or are we going to say we are going to do something about that. Are we going to say we are going to do something about that? I think it is clear we have no idea. It does raise the question. We are not talking about climate change. We are talking about climate change. It is not very big a category. At the same time, it is not a national security threat. And, again, I would say that with the cap, the cap is going to be on this, and this is a national security threat, and it Is going to be a national security threat. So I suggest we dont have the answers and we are going to ignore it. By the way, I am saying it is a national security threat because we are doing something about it. But what we are not doing is addressing climate change. Climate change is a national 

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="Should abortion be illegal?",
              nsamples=1,
              batch_size=1
              )

Should abortion be illegal? Are the laws of most other nations clear? Should we be respecting their wishes? What about their religious beliefs? The question is, in this country, the answer is not only no. No, there are no laws or even judicial precedents on the issue of abortion, they dont even have any. The question is, in this country, what are the areas of the law that are clear? I am not the only one to question whether the U.S, should have the right to allow a woman who has had her womb destroyed to have the right to choose her own life, I havent seen any. The U.S, has the right to have the right to choose a child within a family, to choose a mother or father, to choose who does the household chores. The U.S, should have the right to make sure that their own family is taken care of. The U.S, should be able to make a decision on whether or not they want to have a child within their own family, I dont think that is the way the U.S, should be.
Mr. Speaker, I yield myself the balance 

In [ ]:
gpt2.generate(sess,
              length=250,
              temperature=0.7,
              prefix="Should we have universal healthcare?",
              nsamples=1,
              batch_size=1
              )

Should we have universal healthcare? No. But we should have both. So I will be voting for the bill, if they pass it, and I hope to vote against it. So we have a bill, and I hope we can vote on it. And I hope that we can vote on it.
Mr. Speaker, I rise to honor the life and service of our great House colleague. Congressman Robert J. Rogers. He is a native of Savannah. GA, who served in the Georgia House of Representatives for the past two years. Rogers served on the United States House of Representatives for the past 10 years. He served his seat as the majority leader in the Georgia House of Representatives. He was chairman of the Georgia Subcommittee on Health. Public Health. He is a graduate of St. Augustine High School. He was elected to the Georgia House of Representatives in the State legislature in 2003. After that, he served as the majority whip at the state legislature until being elected in 2007. He has served in the Georgia State House of Representatives since 2007. He holds a